## KR260 AWGN DMA Stream


### Plot function for use in this notebook
The first code block below defines a function that we will use for plotting data throughout this notebook. Note that the function has a `n_samples` argument so that we can limit the number of samples to plot. Plotting more than a few thousand samples can be very slow and consume a lot of RAM.

### Requirements
Install the following:

```bash
pip install numpy==1.26.4
pip install ipympl
pip install ipywidgets
pip install jupyter_bokeh
```

# Initilize

#### Bokeh Plot Function

In [10]:
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
import numpy as np
import math


def plot_time(in_signal,out_signal=None):
    t = np.linspace(0,len(in_signal),len(in_signal))
    output_notebook()
    p = figure(title = "Input & Output Signal")
    
    if out_signal is not None:
        p.line(t,out_signal,legend_label="Output Signal",line_color="red",line_width=1)
    p.line(t,in_signal,legend_label="Input Signal",line_color="blue",line_width=3)
    show(p)

#### Load Overlay

In [26]:
from pynq import Overlay
from pynq import allocate
from pynq import MMIO
import pynq.lib.dma
import time

#import Python library from repo:
import sys
sys.path.append('/root/jupyter_notebooks/puch/')
import fpga.py.puch as puch
import fpga.lib.timestamp.sw.timestamp_regmap as timestamp_regmap
import fpga.lib.led_reg.sw.led_regmap as led_regmap
import fpga.lib.AWGN_GNG.sw.awgn_regmap as awgn_regmap


# Load the overlay
overlay = Overlay('../../overlays/KR260_AWGN_DMA_Stream/output/kr260_awgn_dma_stream.bit')

# Assign blocks to short vars
dma          = overlay.axi_dma_0

# Create PYNQ constructor for Consair RegMap
led_module = led_regmap.RegMap(puch.PynqInterface(overlay.led_reg_0.mmio.base_addr,overlay.led_reg_0.mmio.length))
awgn = awgn_regmap.RegMap(puch.PynqInterface(overlay.gng_top_0.mmio.base_addr,overlay.gng_top_0.mmio.length))
timestamp = timestamp_regmap.RegMap(puch.PynqInterface(overlay.Timestamp_0.mmio.base_addr,overlay.Timestamp_0.mmio.length))

##### Print out the register map for DMA and HLS Gain Blocks

##### Display Time Stamp Register
The time stamp is burned into the FPGA during the build process

In [12]:
print("FPGA Build Timestamp:  " + puch.get_timestamp_str(timestamp))

FPGA Build Timestamp:  2025/9/15 23:16:0


##### Toggle USER_LED[1:0] on/off

In [13]:
for i in range(16):
    led_module.user_leds = i%4
    time.sleep(1)

##### Display AWGN Block Parameters

In [27]:
print("Noise Gain = " + str(awgn.awgn_noise_gain))
awgn.awgn_noise_gain = 4
print("Noise Gain = " + str(awgn.awgn_noise_gain))

print("AWGN Enable = " + str(awgn.awgn_enable))
awgn.awgn_enable = 1
print("AWGN Enable = " + str(awgn.awgn_enable))

Noise Gain = 0
Noise Gain = 4
AWGN Enable = 0
AWGN Enable = 1


In [28]:
print("AWGN F_in Format: ("+str(awgn.f_in_bf.f_in_total) + ", " + str(awgn.f_in_bf.f_in_fractional)+")")
print("AWGN F_out Format: ("+str(awgn.f_out_bf.f_out_total) + ", " + str(awgn.f_out_bf.f_out_fractional)+")")
print("AWGN F_awgn Format: ("+str(awgn.f_awgn_bf.f_awgn_total) + ", " + str(awgn.f_awgn_bf.f_awgn_fractional)+")")

AWGN F_in Format: (16, 12)
AWGN F_out Format: (16, 12)
AWGN F_awgn Format: (16, 11)


### DMA Transfer the Sample buffer from ARM Processor to the AWGN FPGA Module, and plot

In [32]:
awgn_gain = 0.5

# Generate input samples
fin_tBits  = awgn.f_in_bf.f_in_total
fin_fBits  = awgn.f_in_bf.f_in_fractional
fout_tBits = awgn.f_out_bf.f_out_total
fout_fBits = awgn.f_out_bf.f_out_fractional
awgn_tBits = awgn.f_awgn_bf.f_awgn_total
awgn_fBits = awgn.f_awgn_bf.f_awgn_fractional

#n = 1024
#samples = np.ones(n) * 2**(fin_fBits)
#samples = samples.astype(np.int32)

# Create Sine Wave
T = 0.00002
# Sampling frequency
fs = 100e6
# Number of samples
n = int(T * fs)
# Time vector in seconds
t = np.linspace(0, T, n, endpoint=False)
# Samples of the signal
samples = np.sin(0.2e6*2*np.pi*t) 
# Convert samples to 32-bit integers
print('Number of samples: ',len(samples))
samples = samples * 2**(fin_fBits)
samples = samples.astype(np.int32)
#plot_time(samples)

# Read Gain Register
awgn.awgn_noise_gain = int(awgn_gain*(2**awgn_fBits))
#awgn.write(awgn_regmap.RegMap.AWGN_NOISE_GAIN_ADDR,int(awgn_gain*(2**awgn_fBits)))


#awgn_en_reg = awgn.read(awgn_regmap.RegMap.AWGN_ENABLE_ADDR)

print("AWGN Gain           : " + str(awgn.awgn_noise_gain/(2**awgn_fBits)))
print("AWGN Enable         : " + str(awgn.awgn_enable_bf.awgn_enable))
print("DMA Transfer Size   : " + str(len(samples)))
print("AWGN I Sat          : " + str(awgn.awgn_enable_bf.sat_i_ch))
print("AWGN Q Sat          : " + str(awgn.awgn_enable_bf.sat_q_ch))

import pynq
# Allocate buffers for the input and output signals
in_buffer = allocate(shape=(len(samples),), dtype=np.int32)
out_buffer = allocate(shape=(len(samples),), dtype=np.int32)

# Copy the samples to the in_buffer
np.copyto(in_buffer,samples)

# Trigger the DMA transfer and wait for the result
import time
start_time = time.time()
dma.sendchannel.transfer(in_buffer)
#print('Submit sending the transfer into in_buffer')
dma.recvchannel.transfer(out_buffer)
#print('Submit receiving the transfer from out_buffer')
dma.sendchannel.wait()
#print('Done Sending the transfer to in_buffer')
dma.recvchannel.wait()
#print('Done receiving the transfer from out_buffer')
stop_time = time.time()
hw_exec_time = stop_time-start_time

print('DMA Transfer Execution Time  :',hw_exec_time,' sec')

# Plot to the notebook
#plot_time(t,samples,2000,out_signal=out_buffer)

# Free the buffers
in_buffer.close()
out_buffer.close()
t = (out_buffer & 0xFFFF)
out_fp = []
for x in t:
    if(x>(2**(fout_tBits-1))):
          x = x - (2**(fout_tBits))
    out_fp.append(x/(2**fout_fBits))
plot_time(samples/(2**fout_fBits),out_fp)

# Display Status
print("AWGN I Sat          : " + str(awgn.awgn_enable_bf.sat_i_ch))
print("AWGN Q Sat          : " + str(awgn.awgn_enable_bf.sat_q_ch))

print("AWGN I Sat          : " + str(awgn.awgn_enable_bf.sat_i_ch))
print("AWGN Q Sat          : " + str(awgn.awgn_enable_bf.sat_q_ch))


awgn.cnt_ctrl_bf.capture_cnt = 1
print("TVALID Count        : " + str(awgn.tvalid_cnt))
print("TLAST Count         : " + str(awgn.tlast_cnt))

awgn.cnt_ctrl_bf.clear_cnt = 1
print("TVALID Count        : " + str(awgn.tvalid_cnt))
print("TLAST Count         : " + str(awgn.tlast_cnt))


Number of samples:  2000
AWGN Gain           : 0.5
AWGN Enable         : 1
DMA Transfer Size   : 2000
AWGN I Sat          : 0
AWGN Q Sat          : 0
DMA Transfer Execution Time  : 0.0016071796417236328  sec


Loading BokehJS ...

AWGN I Sat          : 0
AWGN Q Sat          : 0
AWGN I Sat          : 0
AWGN Q Sat          : 0
TVALID Count        : 2000
TLAST Count         : 1
TVALID Count        : 0
TLAST Count         : 0


In [15]:
print(np.__version__)


1.26.4


### Read TVALID & TLAST Counters

In [30]:
awgn.cnt_ctrl_bf.capture_cnt = 1
print("TVALID Count    : " + str(awgn.tvalid_cnt))
print("TLAST Count     : " + str(awgn.tlast_cnt))

awgn.cnt_ctrl_bf.clear_cnt = 1
print("TVALID Count    : " + str(awgn.tvalid_cnt))
print("TLAST Count     : " + str(awgn.tlast_cnt))

TVALID Count    : 2000
TLAST Count     : 1
TVALID Count    : 0
TLAST Count     : 0
